### DATA PREPROCESSING

In [ ]:
# install missing dependences


In [2]:


# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample

from db_connection import get_engine
engine = get_engine()

print('Done!')

Done!


In [3]:
print(" Data preprocessing")
print("==" * 40)

# Load data 
loaded_data = pd.read_sql('SELECT * FROM customers', engine)
churn_data = loaded_data.copy()

# display initial data info
print("\n Initial info:")
print(f" - Rows: {churn_data.shape[0]:,}")
print(f" - Columns: {churn_data.shape[1]}")
print(f" - Columns with missing values: {churn_data.isnull().sum().sum()}")

 Data preprocessing

 Initial info:
 - Rows: 10,000
 - Columns: 14
 - Columns with missing values: 0


In [4]:
display(churn_data.head())

print(churn_data.shape)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


(10000, 14)


### Final check for missing value before start trainig

In [4]:
from transform import handle_missing

# Handle missing values with the simple function

churn_data_clean = handle_missing.handle_missing(churn_data, show_info=True)

 CHECKING FOR MISSING VALUES
WoW! There is NO missing values found


### ENCODE CATEGORICAL VARIABLES

In [13]:

# print("\n categorical data encoding")
# print("==" * 40)

# identify categorical columns
# categorical_cols = churn_data_clean.select_dtypes(include=['object']).columns.tolist()
# print(f" Found {len(categorical_cols)} categorical columns:")
# for col in categorical_cols:
#     unique_count = churn_data_clean[col].nunique()
#     unique_values = churn_data_clean[col].unique()[:5]  # just show first 5 values
#     print(f"   => {col}: {unique_count} unique values → {list(unique_values)}...")

# # Create a copy before encoding for reference

# churn_data_original = churn_data_clean.copy()

# print("\n")
# # Handle 'Gender' binary Label Encoding
# if 'Gender' in categorical_cols:
#     le_gender = LabelEncoder()
#     churn_data_clean['Gender'] = le_gender.fit_transform(churn_data['Gender'])
#     print(f" 'Gender' encoded: {dict(zip(le_gender.classes_, range(len(le_gender.classes_))))}")

# # Handle 'Geography' => multiple categories: use One-Hot Encoding
# if 'Geography' in categorical_cols:
#     # Create one-hot encoded columns
#     geo_dummies = pd.get_dummies(churn_data_clean['Geography'], prefix='Geo', drop_first=False)
    
#     # Add to dataset
#     churn_data_clean = pd.concat([churn_data_clean, geo_dummies], axis=1)
    
#     # Drop original column
#     churn_data_clean = churn_data_clean.drop('Geography', axis=1)
    
#     print(f" 'Geography' one-hot encoded: {list(geo_dummies.columns)}")
# print("\n")

### Drop other non-predictive categorical columns

In [5]:

# Drop other non-predictive categorical columns
columns_to_drop =  ['rownumber', 'customerid', 'surname']  # Non-predictive
for col in columns_to_drop:
    if col in churn_data_clean.columns:
        churn_data_clean = churn_data_clean.drop(col, axis=1)
        print(f" Dropped '{col}' (non-predictive)")

# Display encoded data info
print(f"\n After encoding: {churn_data_clean.shape[1]} columns")

 Dropped 'rownumber' (non-predictive)
 Dropped 'customerid' (non-predictive)
 Dropped 'surname' (non-predictive)

 After encoding: 11 columns


###  Dataframe after Deleting non -predictive Columns

In [6]:
display(churn_data_clean.head())
print(f"\n rows: {churn_data_clean.shape[0]}")
print(f"\n columns: {churn_data_clean.shape[1]}")

,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0



 rows: 10000

 columns: 11


### Save `CSV` file to use in AutoML

In [7]:
churn_data_clean.to_csv('data/churn_predictive_data.csv', index=False)

print("Done!")

Done!


### AutoML : SPLIT DATA FOR PYCARET

In [9]:

# SPLIT DATA FOR PYCARET: PyCaret needs train/test split

print("\n train /test split for pycaret")

# define the target column
target_column = 'exited'

# create train and test split
train_data, test_data = train_test_split(
    churn_data_clean,
    test_size=0.2,
    random_state=42,
    stratify=churn_data_clean[target_column]
)

print(f"============================Dataset Split (80/20)===========================")
print(f"   - Training set: {train_data.shape[0]:,} rows")
print(f"   - Test set: {test_data.shape[0]:,} rows")


 train /test split for pycaret
============================Dataset Split (80/20)===========================
   - Training set: 8,000 rows
   - Test set: 2,000 rows


### CLASS IMBALANCE ANALYSIS

In [10]:
# Calculate churn rates
train_churn_rate = train_data[target_column].mean() * 100
test_churn_rate = test_data[target_column].mean() * 100

print(f"Churn Rates:")
print(f"   - Training set: {train_churn_rate:.1f}% churn")
print(f"   - Test set: {test_churn_rate:.1f}% churn")


print(f"\ntrain Data stayed: {train_data[target_column].value_counts()[0]}")
print(f"train data exited: {train_data[target_column].value_counts()[1]}\n")

print(f"   - Imbalance ratio: {train_data[target_column].value_counts()[0]/train_data[target_column].value_counts()[1]:.1f}:1")

# Check if imbalance is severe
imbalance_ratio = train_data[target_column].value_counts()[0] / train_data[target_column].value_counts()[1]
if imbalance_ratio > 3:
    print(f"\n Note: Significant imbalance detected")
    print("   PyCaret options:")
    print("   1. Use 'fix_imbalance=True' in setup()")
    print("   2. Use SMOTE within PyCaret")
else:
    print(f"\n Moderate imbalance - PyCaret can handle this")

Churn Rates:
   - Training set: 20.4% churn
   - Test set: 20.3% churn

train Data stayed: 6370
train data exited: 1630

   - Imbalance ratio: 3.9:1

 Note: Significant imbalance detected
   PyCaret options:
   1. Use 'fix_imbalance=True' in setup()
   2. Use SMOTE within PyCaret


###  DATA SUMMARY FOR PYCARET

In [11]:
print(" DATA READY FOR PYCARET SETUP")
print("=" * 60)

print(f"""
 DATA SHAPES:
   - Full data: {churn_data_clean.shape}
   - Train data: {train_data.shape}
   - Test data: {test_data.shape}

 TARGET VARIABLE:
   - Column: '{target_column}'
   - Training churn rate: {train_churn_rate:.1f}%
   - Test churn rate: {test_churn_rate:.1f}%

 FEATURES:
   - Numerical: {len(train_data.select_dtypes(include=[np.number]).columns) - 1} columns
   - Categorical: {len(train_data.select_dtypes(include=['object']).columns)} columns

 PYCARET SETUP RECOMMENDATIONS:
   1. Use train_data for setup()
   2. Use 'fix_imbalance=True' 
   3. Handle scaling/encoding with PyCaret automatically
""")


 DATA READY FOR PYCARET SETUP

 DATA SHAPES:
   - Full data: (10000, 11)
   - Train data: (8000, 11)
   - Test data: (2000, 11)

 TARGET VARIABLE:
   - Column: 'exited'
   - Training churn rate: 20.4%
   - Test churn rate: 20.3%

 FEATURES:
   - Numerical: 8 columns
   - Categorical: 2 columns

 PYCARET SETUP RECOMMENDATIONS:
   1. Use train_data for setup()
   2. Use 'fix_imbalance=True' 
   3. Handle scaling/encoding with PyCaret automatically



### SAVE PREPROCESSED DATA

In [12]:
# Save the preprocessed data for PyCaret
train_data.to_csv('data/churn_train_pycaret.csv', index=False)
test_data.to_csv('data/churn_test_pycaret.csv', index=False)

print(" Preprocessed data saved:")
print("   - churn_train_pycaret.csv")
print("   - churn_test_pycaret.csv")

print("\n" + "==" * 40)
print(" Ready for PyCaret AutoML!")
print("==" * 40)

 Preprocessed data saved:
   - churn_train_pycaret.csv
   - churn_test_pycaret.csv

 Ready for PyCaret AutoML!
